In [ ]:
import selenium
from selenium import webdriver
import os
from pathlib import Path
import random

In [ ]:
driver = webdriver.Chrome()

In [ ]:
parent = os.path.join(Path().absolute(), "Sections")
NUM_CHAPTERS = 23
def make_chapter_folders():
    os.mkdir(parent)
    for chapter_num in range(1, NUM_CHAPTERS + 1):
        chapter_string = str(chapter_num)
        if chapter_num < 10:
            chapter_string = "0" + chapter_string
        chapter_string = "ch" + chapter_string
        chapter_path = os.path.join(parent, chapter_string)
        os.mkdir(chapter_path)
# make_chapter_folders() 

In [ ]:
repeats =  {
 'Big Picture Questions',
 'HISTORIANS’ VOICES',
 'Introduction',
 'Letter to Changchun',
 'MENELIK II',
 'Memoirs',
 'Namondjok and the Lightning Man',
 'Next Steps: For Further Study',
 'REFLECTIONS',
 'Second Thoughts',
 'The History and Description of Africa',
 'The Marvelous Races of the East',
 'The Rainbow Serpent Awakens',
 'WORKING WITH EVIDENCE',
 'What’s the Significance?',
 'Yhi Brings Life to the World',
 'assessment',
 'bibliography',
 'end-of-section',
 'learning-outcomes'
}

In [ ]:
skips = [
 'What’s the Significance?',
 'assessment',
 'bibliography',
 'end-of-section',
 'learning-outcomes', 
 'Next Steps: For Further Study',
 'Second Thoughts',

]

In [ ]:
breaks = [
     'WORKING WITH EVIDENCE',
]

In [ ]:
section_driver = webdriver.Chrome()
def get_text():
    text = section_driver.find_element_by_tag_name("p").text + "\n"
    for p in section_driver.find_elements_by_class_name("indent"):
        text += p.text + "\n"
    return text

def get_definitions():
    definitions = section_driver.find_elements_by_class_name("glossref")
    if len(definitions) < 1:
        return []
    return [definition.get_attribute("href").split("_")[-1] for definition in definitions]

def process_section(contents):
    html = open("section.html", 'w', encoding='utf-8')
    html.write(contents)
    html.close()
    section_driver.get("file:///C:/Dev/Text%20Summarizer/Textbook%20Scraper/section.html")
    return get_text(), get_definitions()

In [ ]:
#from https://stackoverflow.com/questions/295135/turn-a-string-into-a-valid-filename
import unicodedata
import re

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [ ]:
for root, dirs, files in os.walk(".\HTML", topdown=False):
    for name in files:
        path = Path(os.path.join(root, name))
        f = open(path)
        html = open("test.html", 'w')
        text = f.read()
        html.write(text)
        html.close()
        f.close()
        driver.get("file:///C:/Dev/Text%20Summarizer/Textbook%20Scraper/test.html")
        
        driver.execute_script("document.querySelectorAll('.semantic-sup').forEach(e => e.remove());");
        sections = driver.find_elements_by_tag_name("section")
        
        chapter_number = path.parent.name
        for section in sections:
            if section.get_attribute("class") == "sect1":
                contents = section.get_attribute("innerHTML").split("<section")[0]
            else:
                contents = section.get_attribute("innerHTML")
            title = section.get_attribute("title")
            if title in skips:
                continue
            if title in breaks:
                break
            section_path = Path(os.path.join(Path().absolute(), "Sections", path.parent.name, slugify(title)+ ".txt"))
            section_file = open(section_path, "w+")
            text, defintions = process_section(contents)
            section_file.write(text)
            section_file.close()